# Information Retrieval I #
## Assignment 2: retrieval models [100 points + 10 bonus points] ##
**TA**: Christophe Van Gysel (cvangysel@uva.nl; C3.258B, Science Park 904)

**Secondary TAs**: Harrie Oosterhuis, Nikos Voskarides

In this assignment you will get familiar with basic information retrieval concepts. You will implement and evaluate different information retrieval ranking models and evaluate their performance.

We provide you with a VirtualBox image that comes pre-loaded with an index and a Python installation. To query the index, you'll use a Python package ([pyndri](https://github.com/cvangysel/pyndri)) that allows easy access to the underlying document statistics.

For evaluation you'll use the [TREC Eval](https://github.com/usnistgov/trec_eval) utility, provided by the National Institute of Standards and Technology of the United States. TREC Eval is the de facto standard way to compute Information Retrieval measures and is frequently referenced in scientific papers.

This is a **groups-of-two assignment**, the deadline is **23:59 - 25 January, 2017**. Code quality, informative comments and convincing analysis of the results will be considered when grading. Submission should be done through blackboard, questions can be asked on the course [Piazza](https://piazza.com/class/ixoz63p156g1ts).

### Technicalities (must-read!) ###
This assignment comes pre-loaded on a VirtualBox running Ubuntu. We have configured the indexing software and Python environment such that it works out of the box. You are allowed to extract the files from the VirtualBox and set-up your own non-virtualized environment. However, in this case you are on your own w.r.t. software support.

The assignment directory is organized as follows:
   * `./assignment.ipynb` (this file): the description of the assignment.
   * `./index/`: the index we prepared for you.
   * `./ap_88_90/`: directory with ground-truth and evaluation sets:
      * `qrel_test`: test query relevance collection (**test set**).
      * `qrel_validation`: validation query relevance collection (**validation set**).
      * `topics_title`: semicolon-separated file with query identifiers and terms.
      
`Python + Jupyter`, `Indri`, `Gensim` and `Pyndri` come pre-installed (see `$HOME/.local`). TREC Eval can be found in `$HOME/Downloads/trec_eval.9.0`. The password of the `student` account on the VirtualBox is `datascience`.

### TREC Eval primer ###
The TREC Eval utility can be downloaded and compiled as follows:

    git clone https://github.com/usnistgov/trec_eval.git
    cd trec_eval
    make

TREC Eval computes evaluation scores given two files: ground-truth information regarding relevant documents, named *query relevance* or *qrel*, and a ranking of documents for a set of queries, referred to as a *run*. The *qrel* will be supplied by us and should not be changed. For every retrieval model (or combinations thereof) you will generate a run of the top-1000 documents for every query. The format of the *run* file is as follows:

    $query_identifier Q0 $document_identifier $rank_of_document_for_query $query_document_similarity $run_identifier
    
where
   * `$query_identifier` is the unique identifier corresponding to a query (usually this follows a sequential numbering).
   * `Q0` is a legacy field that you can ignore.
   * `$document_identifier` corresponds to the unique identifier of a document (e.g., APXXXXXXX where AP denotes the collection and the Xs correspond to a unique numerical identifier).
   * `$rank_of_document_for_query` denotes the rank of the document for the particular query. This field is ignored by TREC Eval and is only maintained for legacy support. The ranks are computed by TREC Eval itself using the `$query_document_similarity` field (see next). However, it remains good practice to correctly compute this field.
   * `$query_document_similarity` is a score indicating the similarity between query and document where a higher score denotes greater similarity.
   * `$run_identifier` is an identifier of the run. This field is for your own convenience and has no purpose beyond bookkeeping.
   
For example, say we have two queries: `Q1` and `Q2` and we rank three documents (`DOC1`, `DOC2`, `DOC3`). For query `Q1`, we find the following similarity scores `score(Q1, DOC1) = 1.0`, `score(Q1, DOC2) = 0.5`, `score(Q1, DOC3) = 0.75`; and for `Q2`: `score(Q2, DOC1) = -0.1`, `score(Q2, DOC2) = 1.25`, `score(Q1, DOC3) = 0.0`. We can generate run using the following snippet:

In [168]:
import logging
import sys

def write_run(model_name, data, out_f,
              max_objects_per_query=sys.maxsize,
              skip_sorting=False):
    """
    Write a run to an output file.
    Parameters:
        - model_name: identifier of run.
        - data: dictionary mapping topic_id to object_assesments;
            object_assesments is an iterable (list or tuple) of
            (relevance, object_id) pairs.
            The object_assesments iterable is sorted by decreasing order.
        - out_f: output file stream.
        - max_objects_per_query: cut-off for number of objects per query.
    """
    for subject_id, object_assesments in data.items():
        if not object_assesments:
            logging.warning('Received empty ranking for %s; ignoring.',
                            subject_id)

            continue

        # Probe types, to make sure everything goes alright.
        # assert isinstance(object_assesments[0][0], float) or \
        #     isinstance(object_assesments[0][0], np.float32)
        assert isinstance(object_assesments[0][1], str) or \
            isinstance(object_assesments[0][1], bytes)

        if not skip_sorting:
            object_assesments = sorted(object_assesments, reverse=True)

        if max_objects_per_query < sys.maxsize:
            object_assesments = object_assesments[:max_objects_per_query]

        if isinstance(subject_id, bytes):
            subject_id = subject_id.decode('utf8')

        for rank, (relevance, object_id) in enumerate(object_assesments):
            if isinstance(object_id, bytes):
                object_id = object_id.decode('utf8')

            out_f.write(
                '{subject} Q0 {object} {rank} {relevance} '
                '{model_name}\n'.format(
                    subject=subject_id,
                    object=object_id,
                    rank=rank + 1,
                    relevance=relevance,
                    model_name=model_name))
            
# The following writes the run to standard output.
# In your code, you should write the runs to local
# storage in order to pass them to trec_eval.
write_run(
    model_name='example',
    data={
        'Q1': ((1.0, 'DOC1'), (0.5, 'DOC2'), (0.75, 'DOC3')),
        'Q2': ((-0.1, 'DOC1'), (1.25, 'DOC2'), (0.0, 'DOC3')),
    },
    out_f=sys.stdout,
    max_objects_per_query=1000)

Q2 Q0 DOC2 1 1.25 example
Q2 Q0 DOC3 2 0.0 example
Q2 Q0 DOC1 3 -0.1 example
Q1 Q0 DOC1 1 1.0 example
Q1 Q0 DOC3 2 0.75 example
Q1 Q0 DOC2 3 0.5 example


Now, imagine that we know that `DOC1` is relevant and `DOC3` is non-relevant for `Q1`. In addition, for `Q2` we only know of the relevance of `DOC3`. The query relevance file looks like:

    Q1 0 DOC1 1
    Q1 0 DOC3 0
    Q2 0 DOC3 1
    
We store the run and qrel in files `example.run` and `example.qrel` respectively on disk. We can now use TREC Eval to compute evaluation measures. In this example, we're only interested in Mean Average Precision and we'll only show this below for brevity. However, TREC Eval outputs much more information such as NDCG, recall, precision, etc.

    $ trec_eval -m all_trec -q example.qrel example.run | grep -E "^map\s"
    > map                   	Q1	1.0000
    > map                   	Q2	0.5000
    > map                   	all	0.7500
    
Now that we've discussed the output format of rankings and how you can compute evaluation measures from these rankings, we'll now proceed with an overview of the indexing framework you'll use.

### Pyndri primer ###
For this assignment you will use [Pyndri](https://github.com/cvangysel/pyndri) [[1](https://arxiv.org/abs/1701.00749)], a python interface for [Indri](https://www.lemurproject.org/indri.php). We have indexed the document collection and you can query the index using Pyndri. We will start by giving you some examples of what Pyndri can do:

First we read the document collection index with Pyndri:

In [15]:
import pyndri
import pyndri.compat
import gensim
import copy
import logging
import time
index = pyndri.Index('index/')


The loaded index can be used to access a collection of documents in an easy manner. We'll give you some examples to get some idea of what it can do, it is up to you to figure out how to use it for the remainder of the assignment.

First let's look at the number of documents, since Pyndri indexes the documents using incremental identifiers we can simply take the lowest index and the maximum document and consider the difference:

In [6]:
print("There are %d documents in this collection." % (index.maximum_document() - index.document_base()))
print(index.maximum_document())
print(index.document_base())
print 


There are 164597 documents in this collection.
164598
1


<function print>

Let's take the first document out of the collection and take a look at it:

In [7]:
example_document = index.document(index.document_base())
# print(example_document)

Here we see a document consists of two things, a string representing the external document identifier and an integer list representing the identifiers of words that make up the document. Pyndri uses integer representations for words or terms, thus a token_id is an integer that represents a word whereas the token is the actual text of the word/term. Every id has a unique token and vice versa with the exception of stop words: words so common that there are uninformative, all of these receive the zero id.

To see what some ids and their matching tokens we take a look at the dictionary of the index:

In [17]:
token2id, id2token, _ = index.get_dictionary()
#print(list(id2token.items())[:15])

[(1, 'new'), (2, 'percent'), (3, 'two'), (4, '1'), (5, 'people'), (6, 'million'), (7, '000'), (8, 'government'), (9, 'president'), (10, 'years'), (11, 'state'), (12, '2'), (13, 'states'), (14, 'three'), (15, 'time')]


Using this dictionary we can see the tokens for the (non-stop) words in our example document:

In [173]:
# print([id2token[word_id] for word_id in example_document[1] if word_id > 0])

The reverse can also be done, say we want to look for news about the "University of Massachusetts", the tokens of that query can be converted to ids using the reverse dictionary:

In [18]:
query_tokens = index.tokenize("University of Massachusetts")
print("Query by tokens:", query_tokens)
query_id_tokens = [token2id.get(query_token,0) for query_token in query_tokens]
print("Query by ids with stopwords:", query_id_tokens)
query_id_tokens = [word_id for word_id in query_id_tokens if word_id > 0]
print("Query by ids without stopwords:", query_id_tokens)
print(token2id.get('university'))

Query by tokens: ['university', '', 'massachusetts']
Query by ids with stopwords: [200, 0, 894]
Query by ids without stopwords: [200, 894]
200


Naturally we can now match the document and query in the id space, let's see how often a word from the query occurs in our example document:

In [10]:
matching_words = sum([True for word_id in example_document[1] if word_id in query_id_tokens])
print("Document %s has %d word matches with query: \"%s\"." % (example_document[0], matching_words, ' '.join(query_tokens)))
print("Document %s and query \"%s\" have a %.01f%% overlap." % (example_document[0], ' '.join(query_tokens),matching_words/float(len(example_document[1]))*100))

Document AP890425-0001 has 13 word matches with query: "university  massachusetts".
Document AP890425-0001 and query "university  massachusetts" have a 2.5% overlap.


While this is certainly not everything Pyndri can do, it should give you an idea of how to use it. Please take a look at the [examples](https://github.com/cvangysel/pyndri) as it will help you a lot with this assignment.

**CAUTION**: Avoid printing out the whole index in this Notebook as it will generate a lot of output and is likely to corrupt the Notebook.

### Parsing the query file
You can parse the query file (`ap_88_89/topics_title`) using the following snippet:

In [19]:
import collections
import io
import logging
import sys

def parse_topics(file_or_files,
                 max_topics=sys.maxsize, delimiter=';'):
    assert max_topics >= 0 or max_topics is None

    topics = collections.OrderedDict()

    if not isinstance(file_or_files, list) and \
            not isinstance(file_or_files, tuple):
        if hasattr(file_or_files, '__iter__'):
            file_or_files = list(file_or_files)
        else:
            file_or_files = [file_or_files]

    for f in file_or_files:
        assert isinstance(f, io.IOBase)

        for line in f:
            assert(isinstance(line, str))

            line = line.strip()

            if not line:
                continue

            topic_id, terms = line.split(delimiter, 1)

            if topic_id in topics and (topics[topic_id] != terms):
                    logging.error('Duplicate topic "%s" (%s vs. %s).',
                                  topic_id,
                                  topics[topic_id],
                                  terms)

            topics[topic_id] = terms

            if max_topics > 0 and len(topics) >= max_topics:
                break

    return topics

# with open('./ap_88_89/topics_title', 'r') as f_topics:
#     print(parse_topics([f_topics]))

### Task 1: Implement and compare lexical IR methods [45 points] ### 

In this task you will implement a number of lexical methods for IR using the **Pyndri** framework. Then you will evaluate these methods on the dataset we have provided using **TREC Eval**.

Use the **Pyndri** framework to get statistics of the documents (term frequency, document frequency, collection frequency; **you are not allowed to use the query functionality of Pyndri**) and implement the following scoring methods in **Python**:

- [TF-IDF](http://nlp.stanford.edu/IR-book/html/htmledition/tf-idf-weighting-1.html). **[5 points]**
- [BM25](http://nlp.stanford.edu/IR-book/html/htmledition/okapi-bm25-a-non-binary-model-1.html) with k1=1.2 and b=0.75. **[5 points]**
- Language models ([survey](https://drive.google.com/file/d/0B-zklbckv9CHc0c3b245UW90NE0/view))
    - Jelinek-Mercer (explore different values of 𝛌 in the range [0.1, 0.2, ..., 0.9]). **[5 points]**
    - Dirichlet Prior (explore different values of 𝛍 [500, 1000, ..., 2000]). **[5 points]**
    - Absolute discounting (explore different values of 𝛅 in the range [0.1, 0.2, ..., 0.9]). **[5 points]**
    - [Positional Language Models](http://sifaka.cs.uiuc.edu/~ylv2/pub/sigir09-plm.pdf) define a language model for each position of a document, and score a document based on the scores of its PLMs. The PLM is estimated based on propagated counts of words within a document through a proximity-based density function, which both captures proximity heuristics and achieves an effect of “soft” passage retrieval. Implement the PLM, all five kernels, but only the Best position strategy to score documents. Use 𝛔 equal to 50, and Dirichlet smoothing with 𝛍 optimized on the validation set (decide how to optimize this value yourself and motivate your decision in the report). **[10 points]**
    
Implement the above methods and report evaluation measures (on the test set) using the hyper parameter values you optimized on the validation set (also report the values of the hyper parameters). Use TREC Eval to obtain the results and report on `NDCG@10`, Mean Average Precision (`MAP@1000`), `Precision@5` and `Recall@1000`.

For the language models, create plots showing `NDCG@10` with varying values of the parameters. You can do this by chaining small scripts using shell scripting (preferred) or execute trec_eval using Python's `subprocess`.

Compute significance of the results using a [two-tailed paired Student t-test](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html) **[10 points]**. Be wary of false rejection of the null hypothesis caused by the [multiple comparisons problem](https://en.wikipedia.org/wiki/Multiple_comparisons_problem). There are multiple ways to mitigate this problem and it is up to you to choose one.

Analyse the results by identifying specific queries where different methods succeed or fail and discuss possible reasons that cause these differences.

**NOTE**: Don’t forget to use log computations in your calculations to avoid underflows. 

** Inverted Index List **

In [12]:
import math
from itertools import islice

def take_slice(n, iterable):
    return list(islice(iterable, n))


inverted_list = {}
unique_query_toks = []
query_list = []
n = 20 # check only 20 documents (should be index.maximum_document() - index.document_base())

with open('./ap_88_89/topics_title', 'r') as f_topics:   
    for query in parse_topics([f_topics]).items():
        query_list.append(query)
        
# getting the inverted index list
nr = 0
for query in query_list:
    nr += 1
    if nr == 20: # only check 20 queries
        break
        
    # getting the query term ids
    query_id = query[0]
    query_string = query[1]
    query_tokens = index.tokenize(query_string)
    query_id_tokens = [token2id.get(query_token,0) for query_token in query_tokens]
    query_id_tokens = [word_id for word_id in query_id_tokens if word_id > 0]
    

    for qti in query_id_tokens: # for every query term
        if qti not in unique_query_toks: # only check unique query token once
            inverted_list[qti] = []
            for i in range(1,n+1):
                if i % 50000 == 0:
                    print(i)
                for dti in (dti for dti in index.document(i)[1] if dti > 0):# for all document tokens (not 0)
                    if qti == dti: # compare equality
                        inverted_list[qti] += [index.document_ids([index.document(i)[0]])[0][1]] # add document to query tok id 
                        break

## TF-IDF

In [178]:
def tf(x,t):     
    if isinstance(t, str):
        t = token2id.get(t,0)

    tf = 0
    for word in (word for word in x if word == t):
        tf += 1
            
    return tf     

In [179]:
def idf(t):
    if isinstance(t, str):
        t = token2id.get(t,0)
    return math.log(float(n))-math.log(len(inverted_list[float(t)]))

In [180]:
def tf_idf(t, d):
    if isinstance(d, str):
        d = index.document_ids([d])[0][1]
    
    return math.log(1+tf(index.document(d)[1],t)) * idf(t)

tf_idf(213, 19)

1.7681482684484517

## BM25

In [181]:
def average_length():
    l = 0
    for i in range(1,n+1):
        l += len(index.document(i)[1])
    return l/float(n)

l_av = average_length()

In [182]:
def w(t,d,q):
    k1 = 1.2
    k3 = 1.2
    b = 0.75
    query_tokens = index.tokenize(q)
    query_id_tokens = [token2id.get(query_token,0) for query_token in query_tokens]
    query_id_tokens = [word_id for word_id in query_id_tokens if word_id > 0]
    first = ((k1+1)* tf(index.document(d)[1],t)) / (k1*((1-b)+b*(len(index.document(d)[1])/l_av))+tf(index.document(d)[1],t))
    second = ((k3+1)*tf(query_id_tokens,t))/(k3*tf(query_id_tokens, t))
    return  first*second*idf(t)

In [183]:
w(523, 2,'this is awesome nice nicaraguan wimp role')


2.682185671056863

**Background probability**


In [184]:
def background_prob(w):
    tf_w_C = 0
    for i in range(1,n+1):
        tf_w_C += tf(index.document(i)[1],w) 
    return tf_w_C

## Jelinek-Mercer


In [185]:
# C = entire document collection
def jelinek_mercer(lamb, d, w):
    if isinstance(w, str):
        w = token2id.get(w,0)
    if isinstance(d, str):
        d = index.document_ids([d])[0][1]

    P = (lamb * (tf(index.document(d)[1],w)/len(index.document(d)[1]))) + ((1-lamb) * background_prob(w))
    return P

In [186]:
# [0.1, 0.2, ..., 0.9]

print('0.1: ', round(jelinek_mercer(0.1, 3, 'good'),3))
print('0.2: ', round(jelinek_mercer(0.2, 3, 'good'),3))
print('0.3: ', round(jelinek_mercer(0.3, 3, 'good'),3))
print('0.4: ', round(jelinek_mercer(0.4, 3, 'good'),3))
print('0.5: ', round(jelinek_mercer(0.5, 3, 'good'),3))
print('0.6: ', round(jelinek_mercer(0.6, 3, 'good'),3))
print('0.7: ', round(jelinek_mercer(0.7, 3, 'good'),3))
print('0.8: ', round(jelinek_mercer(0.8, 3, 'good'),3))
print('0.9: ', round(jelinek_mercer(0.9, 3, 'good'),3))


0.1:  4.5
0.2:  4.0
0.3:  3.5
0.4:  3.0
0.5:  2.5
0.6:  2.0
0.7:  1.5
0.8:  1.0
0.9:  0.5


## Dirichlet Prior

In [187]:
def dirichlet_prior(mu,d,w):
    if isinstance(d, str):
        d = index.document_ids([d])[0][1]
    if isinstance(w, str):
        w = token2id.get(w,0)    
    
       
    
    return ((len(index.document(d)[1])/(len(index.document(d)[1])+mu)) * (tf(index.document(d)[1],w)/len(index.document(d)[1]))) + ((mu/(mu+len(index.document(d)[1]))) * background_prob(w))

In [188]:
# 𝛍 [500, 1000, ..., 2000]

print('500: ', round(dirichlet_prior(500, 3, 'good'),3))
print('1000: ', round(dirichlet_prior(1000, 3, 'good'),3))
print('1500: ', round(dirichlet_prior(1500, 3, 'good'),3))
print('2000: ', round(dirichlet_prior(2000, 3, 'good'),3))


500:  4.065
1000:  4.484
1500:  4.644
2000:  4.728


## Absolute Discounting

In [189]:
def absolute_discounting(delta, d, w):
    unique = []
    for word in index.document(d)[1]:
        if word not in unique:
            unique.append(word)
    return (max(tf(index.document(d)[1],w)-delta, 0)/len(index.document(d)[1])) + (((delta * len(unique))/len(index.document(d)[1])) * (background_prob(w)))

In [190]:
# delta = [0.1, 0.2, ..., 0.9]

print('0.1: ', round(absolute_discounting(0.1, 3, 'good'),3))
print('0.2: ', round(absolute_discounting(0.2, 3, 'good'),3))
print('0.3: ', round(absolute_discounting(0.3, 3, 'good'),3))
print('0.4: ', round(absolute_discounting(0.4, 3, 'good'),3))
print('0.5: ', round(absolute_discounting(0.5, 3, 'good'),3))
print('0.6: ', round(absolute_discounting(0.6, 3, 'good'),3))
print('0.7: ', round(absolute_discounting(0.7, 3, 'good'),3))
print('0.8: ', round(absolute_discounting(0.8, 3, 'good'),3))
print('0.9: ', round(absolute_discounting(0.9, 3, 'good'),3))

0.1:  0.243
0.2:  0.487
0.3:  0.73
0.4:  0.974
0.5:  1.217
0.6:  1.461
0.7:  1.704
0.8:  1.948
0.9:  2.191


## Positional Language Models

### Task 2: Latent Semantic Models (LSMs) [25 points + 10 bonus points] ###

In this task you will experiment with applying a distributional semantics methods ([word2vec](http://arxiv.org/abs/1411.2738)  **[5 points]**, [LSI](http://lsa3.colorado.edu/papers/JASIS.lsi.90.pdf) **[5 points]**, [LDA](https://www.cs.princeton.edu/~blei/papers/BleiNgJordan2003.pdf) **[5 points]** and [doc2vec](https://cs.stanford.edu/~quocle/paragraph_vector.pdf) **[5 points]**) for retrieval.

You do not need to implement word2vec, LSI, LDA and doc2vec on your own. Instead, you can use [gensim](http://radimrehurek.com/gensim/index.html) (pre-loaded on the VirtualBox). An example on how to integrate Pyndri with Gensim for word2vec can be found [here](https://github.com/cvangysel/pyndri/blob/master/examples/word2vec.py). For the remaining latent vector space models, you will need to implement connector classes (such as `IndriSentences`) by yourself.

In order to use a latent semantic model for retrieval, you need to:
   * build a representation of the query **q**,
   * build a representation of the document **d**,
   * calculate the similarity between **q** and **d** (e.g., cosine similarity, KL-divergence).
     
The exact implementation here depends on the latent semantic model you are using. For example, in the case of word2vec, you only have vectors for individual words and not for documents or phrases. Try one of the following methods for producing these representations:
   * Average or sum the word vectors.
   * Cluster words in the document using [k-means](http://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html) and use the centroid of the most important cluster. Experiment with different values of K for k-means.
   * Using the [bag-of-word-embeddings representation](https://ciir-publications.cs.umass.edu/pub/web/getpdf.php?id=1248). **[10 bonus points]**
   
Each of these LSMs come with various hyperparameters to tune. Make a choice on the parameters, and explicitly mention the reasons that led you to these decisions. You can use the validation set to optimize hyper parameters you see fit; motivate your decisions. In addition, mention clearly how the query/document representations were constructed for each LSM and explain your choices.

In this experiment, you will first obtain an initial top-1000 ranking for each query using TF-IDF in **Task 1**, and then re-rank the documents using the LSMs. Use TREC Eval to obtain the results and report on `NDCG@10`, Mean Average Precision (`MAP@1000`), `Precision@5` and `Recall@1000`.

Perform significance testing **[5 points]** (similar as in Task 1) in the class of semantic matching methods.

In [191]:
# Provide right gensim input with pyndri for word2vec, this code is provided with the assignment.
# IndriSentences gives sentences in the right format for word2vec.
# It works as a bridge between gensim and pyndri -> we have to do this for the other distributional semantic methods as well.
# Understanding which input parameters in what format that are needed for each model shows understanding of the model itself.

import pyndri.compat
import gensim
import copy
import logging
word2vec_init = gensim.models.Word2Vec(
    size=64,  # Embedding size - initially 300
    window=5,  # One-sided window size, init was 5
    sg=True,  # Skip-gram.
    min_count=1,  # Minimum word frequency.
    sample=1e-3,  # Sub-sample threshold.
    hs=False,  # Hierarchical softmax.
    negative=10,  # Number of negative examples.
    iter=1,  # Number of iterations.
    workers=8,  # Number of workers.
)

dictionary = pyndri.extract_dictionary(index)
sentences = pyndri.compat.IndriSentences(index, dictionary, max_documents =1) # max_documents due to time complexity

logging.info('Constructing word2vec vocabulary.')

# Build vocab.
word2vec_init.build_vocab(sentences, trim_rule=None)

models = [word2vec_init]

for epoch in range(1, 6):#range(1, 5 + 1):
    logging.info('Epoch %d', epoch)
    model = copy.deepcopy(models[-1])
    model.train(sentences)

    models.append(model)

logging.info('Trained models: %s', models)
print(models[-1]) # last model is most advanced

### END OF CODE AS PROVIDED BY ASSIGNMENT



Word2Vec(vocab=195, size=64, alpha=0.025)


In [192]:
class IndriBOW(gensim.interfaces.CorpusABC):

    def __init__(self, index, dictionary, max_documents=None):
        assert isinstance(index, pyndri.Index)

        self.index = index
        self.dictionary = dictionary

        self.max_documents = max_documents

    def _maximum_document(self):
        if self.max_documents is None:
            return self.index.maximum_document()
        else:
            return min(
                self.max_documents + self.index.document_base(),
                self.index.maximum_document())

    def __iter__(self): # both used for LSI and LDI
        for int_doc_id in range(self.index.document_base(),
                                self._maximum_document()):
            ext_doc_id, tokens = self.index.document(int_doc_id)

            yield self.dictionary.doc2bow(
                self.dictionary[token_id]
                for token_id in tokens
                if token_id > 0 and token_id in self.dictionary)

    def __len__(self):
        return self._maximum_document() - self.index.document_base()

In [214]:

word2vec_init = gensim.models.Word2Vec(
    size=64,  # Embedding size - initially 300
    window=5,  # One-sided window size, init was 5
    sg=True,  # Skip-gram.
    min_count=1,  # Minimum word frequency.
    sample=1e-3,  # Sub-sample threshold.
    hs=False,  # Hierarchical softmax.
    negative=10,  # Number of negative examples.
    iter=1,  # Number of iterations.
    workers=8,  # Number of workers.
)

dictionary = pyndri.extract_dictionary(index)
start = time.time()
sentences = pyndri.compat.IndriSentences(index, dictionary, max_documents=50000) # max_documents due to time complexity
print('initialised sentences')
print(time.time()-start)

logging.info('Constructing word2vec vocabulary.')

# Build vocab.
word2vec_init.build_vocab(sentences, trim_rule=None)

models = [word2vec_init]
print('going in epoch')
for epoch in range(1, 5+1):#range(1, 5 + 1):
    start = time.time()
    logging.info('Epoch %d', epoch)
    model = copy.deepcopy(models[-1])
    model.train(sentences)

    models.append(model)
    print(time.time()-start)
logging.info('Trained models: %s', models)
print(models[-1]) # last model is most advanced
models[-1].save('Small_Morning_Run')
savedW2Vmodel = gensim.models.Word2Vec.load('Small_Morning_Run')
print(savedW2Vmodel)


### END OF CODE AS PROVIDED BY ASSIGNMENT



initialised sentences
0.07100343704223633
going in epoch
274.00119137763977
273.1599049568176
6175.089705705643
296.12786197662354
272.2163770198822
Word2Vec(vocab=153153, size=64, alpha=0.025)
Word2Vec(vocab=153153, size=64, alpha=0.025)


In [79]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.cluster import mutual_info_score #is equivalent to KL divergence
import numpy
### Build representation for q and d and calculate cosine similarity and KL divergence.

# Representation of query for Word2Vec (and Doc2Vec)
# Takes average of all query terms in 1 query, so the query is represented by 1 vector
savedW2Vmodel = gensim.models.Word2Vec.load('Small_Morning_Run')
def get_q_rep(index, query, w2vmodel):
    query_tokens = index.tokenize(query[1])
    vectors = []
    for token in query_tokens:
        vector = w2vmodel['token']
        vectors.append(vector)

    avg_query = np.mean(np.array(vectors), axis=0)
    return avg_query, query[0]

avg_q, queryID = get_q_rep(index, 'University of Massachusetts', savedW2Vmodel)

def get_d_rep(document, w2vmodel):
    wordIDs = document[1][:]
    vectors = []
    for wordID in wordIDs:
        if wordID > 0:
            wordstring = id2token[wordID]
            vector = w2vmodel[wordstring]
            vectors.append(vector)

    avg_doc = np.mean(np.array(vectors), axis=0)
    return avg_doc

avg_d = get_d_rep(index.document(1),savedW2Vmodel)

# get cosine similarity: 

from scipy import spatial
cosine = spatial.distance.cosine(avg_q, avg_d)
print(cosine)

cosine_similarity(numpy.reshape(avg_q, (1, -1)),  numpy.reshape(avg_d, (1, -1)))[0][0] 
#mutual_info_score(avg_q, avg_d)


0.348099537217


0.65190053

In [82]:
avg_q, queryID = get_q_rep(index, 'University of Massachusetts', savedW2Vmodel)


def check_cosine_given_query(avg_q, queryID, internal_docIDs):
    word2vec_dict = {}
    avg_ds = []
    for internal_docID in internal_docIDs: # amount of documents
        avg_d = get_d_rep(index.document(internal_docID),savedW2Vmodel)
        avg_ds.append(avg_d)
        cosine = cosine_similarity(numpy.reshape(avg_q, (1, -1)),  numpy.reshape(avg_d, (1, -1)))[0][0] 
        external_ID = index.document(internal_docID)[0]
        word2vec_dict[external_ID] = cosine
    return word2vec_dict

cosinesdict = check_cosine_given_query(avg_q, queryID, [1,2,3,4])
print(cosinesdict)

def check_cosine_all_queries(lijstvandiede):
    dic = {}
    for queryid in querylist:
        
        docs = lijstvandiede[queryid]
        
        queryresult = check_cosine_given_query(docs)
        dic[queryid] = make_dict_format(queryresult)
    return dic

{'AP890425-0004': 0.68140054, 'AP890425-0003': 0.71697617, 'AP890425-0002': 0.74149895, 'AP890425-0001': 0.65190053}


In [195]:


# logging.basicConfig(level=logging.INFO)
# if len(sys.argv) <= 1:
#     logging.error('Usage: python %s <path-to-indri-index>'.format(sys.argv[0]))

#     sys.exit(0)

# logging.info('Initializing LSI')


# # with pyndri.open(sys.argv[1]) as index:
# logging.info('Loading vocabulary.')
# print('-1-')
# #dictionary = pyndri.extract_dictionary(index)
# print('-2-')
# sentences = IndriBOW(index, dictionary)
# print('-3-')
# logging.info('Constructing sentences for LSI.')
# print('-4-')
# # Build vocab.

# lsimodel = gensim.models.LsiModel(sentences, 200, dictionary.id2token, 20000)
# print(lsimodel.show_topic(1, topn=1))
# print('-5-')
# # LsiModel.build_vocab(sentences, trim_rule=None)
# print('-6-')
# lsimodel.save('lsi_big_overnight')
# savedlsimodel = gensim.models.LsiModel.load('lsi_big_overnight')
# print(savedlsimodel.show_topic(1, topn=1))


# logging.info('Initializing LDA')


# # with pyndri.open(sys.argv[1]) as index:
# logging.info('Loading vocabulary.')
# #print('-1-')
# #dictionary = pyndri.extract_dictionary(index)
# #print('-2-')
# #sentences = IndriBOW(index, dictionary,max_documents=2)

# logging.info('Constructing sentences for LDA.')
# print('-7-')
# # Build vocab.

# ldamodel = gensim.models.LdaModel(sentences, 64, dictionary.id2token, 20000, distributed=False)
# print(ldamodel.show_topic(1, topn=1))
# print('-8-')
# # LdaModel.build_vocab(sentences, trim_rule=None)
# print('-9-')
# ldamodel.save('lda_big')
# savedldamodel = gensim.models.LdaModel.load('lda_big')
# print(savedldamodel.show_topic(1, topn=1))

# Raises error: 
    
#     ---------------------------------------------------------------------------
# TypeError                                 Traceback (most recent call last)
# <ipython-input-79-b41044639cda> in <module>()
#      43 # Build vocab.
#      44 
# ---> 45 ldamodel = gensim.models.LdaModel(sentences, 64, dictionary.id2token, 20000, distributed=False)
#      46 print(ldamodel.show_topic(1, topn=1))
#      47 print('-8-')

# TypeError: __init__() got multiple values for argument 'distributed'

NameError: name 'index' is not defined

In [197]:
## LSI ##
'''LSI compares how often words appear together in the same document
and compares how often those occurences happen in all of the documents 
that the dictionary has in its index.'''

# Get docs as bag of words #
# print(id2token[0]) is not possible, zeroeth token = irrelevant token. Have to leave this out. xxx
def getdocs(index, max_docs):
    dictionary = pyndri.extract_dictionary(index)
    bowdoc=[]
    if max_docs == None:
        for document in range(index.document_base(), index.maximum_document()):  
            bowdoc.append(dictionary.doc2bow(index.document(document)[1])[1:]) #[1:] throws out irrelevant word '0'
    else:
        for document in range(index.document_base(), index.document_base()+max_docs):  
            bowdoc.append(dictionary.doc2bow(index.document(document)[1])[1:])
    return bowdoc
bowdoc = getdocs(index, 2)

print("got bow")
lsimodels=[]
lsimodel = gensim.models.LsiModel(bowdoc, 2, id2token, 2, 1.0, False, True, 2, 100)


got bow


In [198]:
print(lsimodel.show_topic(1, topn=1))

[('panama', -0.37052153620176137)]


In [199]:
## Latent Dirichlet Allocation ##


ldamodel = gensim.models.ldamodel.LdaModel(bowdoc, 10, id2token, False, 20, 1, 1,'symmetric', None, 0.5, 1.0, 10, 50, 0.001, 0.01, None, {}, 0.01, False)
print(ldamodel.show_topic(1, topn=1))

/usr/local/lib/python3.5/dist-packages/gensim/models/ldamodel.py:528: RuntimeWarning: overflow encountered in exp2
  (perwordbound, np.exp2(-perwordbound), len(chunk), corpus_words))


[('panama', 9.0568516012420351e-05)]


In [200]:
## Doc2Vec ##

#gensim.models.doc2vec.Doc2Vec(documents=None, dm_mean=None, dm=1, dbow_words=0, dm_concat=0, dm_tag_count=1, docvecs=None, docvecs=mapfile=None, comment=None, trim_rule=None, **kwargs)

class IndriDoc2Vec(gensim.interfaces.CorpusABC):

    def __init__(self, index, dictionary, max_documents=None):
        assert isinstance(index, pyndri.Index)

        self.index = index
        self.dictionary = dictionary

        self.max_documents = max_documents

    def _maximum_document(self):
        if self.max_documents is None:
            return self.index.maximum_document()
        else:
            return min(
                self.max_documents + self.index.document_base(),
                self.index.maximum_document())

    def __iter__(self): # Doc2Vec
        for int_doc_id in range(self.index.document_base(),
                                self._maximum_document()):
            ext_doc_id, tokens = self.index.document(int_doc_id)

            yield gensim.models.doc2vec.TaggedDocument(words =[self.dictionary[token_id]
                                                              for token_id in tokens if token_id >0
                                                               and token_id in self.dictionary],
                                                      tags = ['doc_'+str( int_doc_id) ])

    def __len__(self):
        return self._maximum_document() - self.index.document_base()


In [213]:
doc2vec_init = gensim.models.doc2vec.Doc2Vec(
 #   size=size,  # Embedding size - initially 300
  #  window=window,  # One-sided window size, init was 5
    #sg=True,  # Skip-gram.
    min_count=1,  # Minimum word frequency.
    sample=1e-3,  # Sub-sample threshold.
    hs=False,  # Hierarchical softmax.
    negative=10,  # Number of negative examples.
    iter=1,  # Number of iterations.
    workers=8,  # Number of workers.
    dm_mean=None, 
    dm=10, 
    dbow_words=0, 
    dm_concat=0, 
)
dictionary = pyndri.extract_dictionary(index)
start = time.time()
sentences = IndriDoc2Vec(index, dictionary, max_documents=2) # max_documents due to time complexity
print('initialised sentences')
print(time.time()-start)

logging.info('Constructing doc2vec vocabulary.')

# Build vocab.
doc2vec_init.build_vocab(sentences, trim_rule=None)

models = [doc2vec_init]
print('going in epoch')
for epoch in range(1, 5+1):#range(1, 5 + 1):
    start = time.time()
    logging.info('Epoch %d', epoch)
    model = copy.deepcopy(models[-1])
    model.train(sentences)

    models.append(model)
    print(time.time()-start)
logging.info('Trained models: %s', models)
print(models[-1]) # last model is most advanced
models[-1].save('Small_Morning_Run')
savedD2Vmodel = gensim.models.Doc2Vec.load('Small_Morning_Run')
print(savedD2Vmodel)

Doc2Vec_model = gensim.models.Doc2Vec(sentences, size=100, window=8, min_count=5, workers=4)




initialised sentences
0.07341694831848145
going in epoch
0.018000364303588867
0.02876567840576172
0.031171083450317383
0.03630709648132324
0.026875019073486328
Doc2Vec(dbow,d100,n10,s0.001,t8)
Doc2Vec(dbow,d100,n10,s0.001,t8)


### Task 3: Learning to rank (LTR) [10 points] ###

In this task you will get an introduction into learning to rank for information retrieval. You will experiment with a pointwise learning to rank method, logistic regression, implemented in [scikit-learn](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html).

**NOTE**: you can only perform this task if you have completely finished Task 1 and Task 2.

In this experiment, you will use the retrieval methods you implemented in Task 1 and Task 2 as features for the learning to rank model. Train your LTR model using 10-fold cross validation on the test set. For every query, first create a document candidate set using the top-1000 documents using TF-IDF. Secondly, compute query-document values using the retrieval models above and use them as features. Note that the feature values of different retrieval methods are likely to be distributed differently.

Your approach will definitely not be as good as the state-of-the-art since you are taking a pointwise approach, but we do not ask you to try pair- or listwise methods because they will be the main topic of the next assignment.

In [ ]:
from sklearn.linear_model import LogisticRegression

### Task 4: Write a report [20 points; instant FAIL if not provided] ###

The report should be a PDF file created using the [sigconf ACM template](https://www.acm.org/publications/proceedings-template) and will determine a significant part of your grade.

   * It should explain what you have implemented, motivate your experiments and detail what you expect to learn from them. **[10 points]**
   * Lastly, provide a convincing analysis of your results and conclude the report accordingly. **[10 points]**
      * Do all methods perform similarly on all queries? Why?
      * Is there a single retrieval model that outperforms all other retrieval models (i.e., silver bullet)?
      * ...

**Hand in the report and your self-contained implementation source files.** Do not send us the VirtualBox, but only the files that matter, organized in a well-documented zip/tgz file with clear instructions on how to reproduce your results. That is, we want to be able to regenerate all your results with minimal effort. You can assume that the index and ground-truth information is present in the same file system structure as on the VirtualBox.
